In [1]:
import os
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus

In [2]:
class product_info:
    def __init__(self, product, product_full_name, version, language):
        self.product = product
        self.product_full_name = product_full_name
        self.version = version
        self.language = language

In [3]:
products = []
products.append(product_info('red_hat_openshift_ai_self-managed',
                             'Red Hat OpenShift AI Self-Managed',
                             '2.8',
                             'en-US'))

In [4]:
milvus = {}
milvus["MILVUS_HOST"] = "vectordb-milvus.milvus.svc.cluster.local"
milvus["MILVUS_PORT"] = 19530
milvus["MILVUS_USERNAME"] = os.getenv('MILVUS_USERNAME')
milvus["MILVUS_PASSWORD"] = os.getenv('MILVUS_PASSWORD')


In [5]:
model_kwargs = {'trust_remote_code': True, 'device': 'cuda'}
embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs=model_kwargs,
    show_progress=True
)


You try to use a model that was created with version 2.4.0.dev0, however, your version is 2.4.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



<All keys matched successfully>


In [6]:
query = "How do I install this product?"

for product_info in products:
    MILVUS_COLLECTION = f"{product_info.product.replace('-', '_')}_{product_info.language.replace('-', '_')}_{product_info.version.replace('.', '_')}"
    db = Milvus(
        embedding_function=embeddings,
        connection_args={"host": milvus["MILVUS_HOST"], "port": milvus["MILVUS_PORT"], "user": milvus["MILVUS_USERNAME"], "password": milvus["MILVUS_PASSWORD"]},
        collection_name=MILVUS_COLLECTION,
        metadata_field="metadata",
        text_field="page_content",
        auto_id=True,
        drop_old=False
        )
    docs_with_score = db.similarity_search_with_score(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.8957121968269348
Section: Installing and uninstalling OpenShift AI Self-Managed / Chapter 2. Installing and deploying OpenShift AI / 2.3. Installing the Red Hat OpenShift AI Operator / 2.3.2. Installing the Red Hat OpenShift AI Operator by using the web

Content:
8. For Installed Namespace, select `redhat-ods-operator (Operator recommended)`.
9. Under Update approval, select either `Automatic` or `Manual`.
10. Click Install.  
An installation pane opens. When the installation finishes, a checkmark
appears beside the Operator name in the installation pane.  
Verification  
* In the OpenShift Container Platform web console, click Operators → Installed Operators and confirm that the Red Hat OpenShift AI Operator shows one of the following statuses:  
* `Installing` \- installation is in progress; wait for this to change to `Succeeded`. This might take several minutes.
* `Succeeded` \- installation i